In [1]:
!pip install ultralytics==8.1.15

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.4/40.4 kB 1.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 715.1/715.1 kB 18.4 MB/s eta 0:00:0000:01


In [ ]:
!pip install torch

In [2]:
import os
import random
import shutil
from collections import defaultdict, Counter
from pathlib import Path
from typing import Dict, List, Tuple, Optional, Iterable
import yaml

import cv2
import plotly.express as px
from plotly import subplots
from tqdm.notebook import tqdm
import torch
from ultralytics import YOLO

In [3]:
DATASET_PATH = '/kaggle/input/poaching-and-animal-detection-dataset'  # Path to source dataset
MASTER_PATH = '/kaggle/working/' # Path where all outputs are stored (intermediate and final)
DEBUG = False # Activete to run notebook faster
CPU = False

if not CPU:
    assert torch.cuda.is_available(), 'CUDA not found!'

In [4]:
import os
import random
import shutil
from collections import defaultdict, Counter
from pathlib import Path
from tqdm import tqdm
from typing import Dict, List, Optional, Tuple
import cv2
import yaml  # Ensure PyYAML is installed

# Type aliases
DatasetIndex = Dict[str, Dict[str, List[str]]]
DatasetStats = Dict[str, int]

# LookupTable class definition
class LookupTable:
    def __init__(self, add_unknown_token=True):
        self.add_unknown_token = add_unknown_token
        self.table = {}
        self.inverse_table = {}
        if add_unknown_token:
            self.add('<unknown>')

    def add(self, item):
        if item not in self.table:
            index = len(self.table)
            self.table[item] = index
            self.inverse_table[index] = item

    def __getitem__(self, item):
        if self.add_unknown_token:
            return self.table.get(item, self.table['<unknown>'])
        return self.table[item]

    def __len__(self):
        return len(self.table)

    def __iter__(self):
        return iter(self.table)

# AnimalToYOLODatasetAdapter class definition
class AnimalToYOLODatasetAdapter:
    
    def __init__(self, path: str, label_filter: Optional[List[str]] = None):
        self._path = path
        self._index, self.label_stats, self.split_stats, self.label_lookup, self._size = \
            self._index_dataset(path, label_filter)
        
    @staticmethod
    def _index_dataset(path: str, label_filter: Optional[List[str]] = None) \
        -> Tuple[DatasetIndex, DatasetStats, DatasetStats, LookupTable, int]:
        
        index: DatasetIndex = defaultdict(dict)
        label_stats: DatasetStats = Counter()
        split_stats: DatasetStats = Counter()
        lookup = LookupTable(add_unknown_token=False)
        size = 0

        splits = os.listdir(path)
        for split in splits:        
            split_path = os.path.join(path, split)
            if not os.path.isdir(split_path):
                continue
            labels = os.listdir(split_path)
            for label in tqdm(labels, desc=f'Indexing {split}', unit='sample'):
                if label_filter is not None and label not in label_filter:
                    continue
                
                label_path = os.path.join(split_path, label)
                sample_ids = [Path(filename).stem for filename in os.listdir(label_path) 
                              if filename != 'Label' and (filename.endswith('.jpg') or filename.endswith('.jpeg'))]
                annotations_path = os.path.join(label_path, 'Label')
                if not os.path.exists(annotations_path):
                    continue
                annot_sample_ids = [Path(filename).stem for filename in os.listdir(annotations_path)
                                    if filename.endswith('.txt')]
                assert set(sample_ids) == set(annot_sample_ids), 'Image sample ids and annotation sample ids do not match'

                # Update index, stats and lookup
                index[split][label] = sample_ids
                
                n_samples = len(sample_ids)
                label_stats[label] += n_samples
                split_stats[split] += n_samples
                size += n_samples
                
                lookup.add(label)

        return dict(index), dict(label_stats), dict(split_stats), lookup, size
    
    def __len__(self) -> int:
        return self._size
    
    @property
    def labels(self) -> List[str]:
        return list(self.label_lookup)

    @property
    def n_labels(self) -> int:
        return len(self.label_lookup)
    
    def get_random_samples(self, n: int, split: str = 'train') -> List[Tuple[str, str, str]]:
        split_index = self._index.get(split, {})
        if not split_index:
            raise ValueError(f'Split "{split}" not found in the dataset.')
        label_names = self.labels
        
        result: List[Tuple[str, str, str]] = []
        for i in range(n):
            label = random.choice(label_names)
            sample_ids = split_index[label]
            sample_id = random.choice(sample_ids)
            result.append((split, label, sample_id))
            
        return result
    
    def get_split_size(self, split: str) -> int:
        if split not in self.split_stats:
            raise ValueError(f'Split "{split}" not found in the dataset.')
        return self.split_stats[split]

    def get_image_path(self, split: str, label: str, sample_id: str) -> str:
        for ext in ['.jpg', '.jpeg']:
            image_path = os.path.join(self._path, split, label, f'{sample_id}{ext}')
            if os.path.exists(image_path):
                return image_path
        raise FileNotFoundError(f'Image for sample id "{sample_id}" not found in {split}/{label}!')

    def load_image(self, split: str, label: str, sample_id: str) -> str:
        image_path = self.get_image_path(split, label, sample_id)
        if not os.path.exists(image_path):
            raise FileNotFoundError(f'Image "{image_path}" not found!')
        return cv2.imread(image_path)

    def get_annot_path(self, split: str, label: str, sample_id: str) -> str:
        return os.path.join(self._path, split, label, 'Label', f'{sample_id}.txt')

    def parse_annot(self, split: str, label: str, sample_id: str) -> List[Tuple[str, float, float, float, float]]:
        annot_path = self.get_annot_path(split, label, sample_id)
        with open(annot_path, 'r', encoding='utf-8') as f:
            lines = f.readlines()
        annots: List[Tuple[str, float, float, float, float]] = []
        for l in lines:
            items = l.split()
            if len(items) < 5:
                print(f"Skipping invalid annotation in {annot_path}: {l.strip()}")
                continue
            label_name = ' '.join(items[:-4])
            coords = [float(v) for v in items[-4:]]
            annots.append((label_name, *coords))
        return annots
    
    def convert(self, path: str) -> None:
        for split in self._index:
            split_path = os.path.join(path, split)
            images_path = os.path.join(split_path, 'images')
            labels_path = os.path.join(split_path, 'labels')
            Path(images_path).mkdir(parents=True, exist_ok=True)
            Path(labels_path).mkdir(parents=True, exist_ok=True)
            
            for label, sample_ids in tqdm(self._index[split].items(), desc='Converting to Yolo format', total=len(self._index[split])):
                assert len(sample_ids) == len(set(sample_ids))
                for sample_id in sample_ids:
                    image_path = self.get_image_path(split, label, sample_id)
                    new_image_path = os.path.join(images_path, f'{sample_id}.jpg')
                    annots = self.parse_annot(split, label, sample_id)
                    new_annot_path = os.path.join(labels_path, f'{sample_id}.txt')

                    # Debug statement to check the annotations
                    print(f"Annotations for {sample_id}: {annots}")
                    
                    # Image needs to be loaded in order to read width and height
                    # which are required for coordinate normalization
                    image = self.load_image(split, label, sample_id)
                    h, w, _ = image.shape
                    
                    # Conversion
                    converted_annot: List[Tuple[int, float, float, float, float]] = []
                    for label, x_min, y_min, x_max, y_max in annots:
                        label_index = self.label_lookup[label]
                        x_center = (x_min + x_max) / (2 * w)
                        y_center = (y_min + y_max) / (2 * h)
                        width = (x_max - x_min) / w
                        height = (y_max - y_min) / h
                        
                        converted_annot.append((label_index, x_center, y_center, width, height))
                        
                    # Save data
                    with open(new_annot_path, 'a', encoding='utf-8') as f:
                        converted_annot_lines = [' '.join([str(v) for v in row]) for row in converted_annot]
                        f.write('\n'.join(converted_annot_lines))
                        f.write('\n')
                        
                    if not os.path.exists(new_image_path):  
                        shutil.copy(image_path, new_image_path)

# Example usage:
# Make sure to set the path and debug variables
DATASET_PATH = '/kaggle/input/poaching-and-animal-detection-dataset/Project Dataset for Animal Prediction'
MASTER_PATH = '/kaggle/working/'
DEBUG = False

adapter = AnimalToYOLODatasetAdapter(
    path=DATASET_PATH, 
    label_filter=['Horse'] if DEBUG else None
)

print(f'Total number of samples in the dataset is {len(adapter)}.')
print(f'Total number of classes in the dataset is {adapter.n_labels}.')
try:
    print(f'Train dataset size is {adapter.get_split_size("train")} (images). Test dataset size is {adapter.get_split_size("test")} (images)')
except ValueError as e:
    print(e)

adapter.convert(MASTER_PATH)

# Generate the YOLO config file
class_names = list(adapter.label_lookup.table.keys())
config = {
    'path': MASTER_PATH,
    'train': 'train/images',
    'val': 'test/images',
    'nc': len(class_names),  # Number of classes
    'names': class_names
}

# Print the config
print(config)

# Define the path where you want to save the config.yaml file
config_path = '/kaggle/working/config.yaml'

# Write the config dictionary to a YAML file
with open(config_path, 'w') as f:
    yaml.dump(config, f)

# Print a confirmation message
print(f'Config file saved to {config_path}')

Indexing train: 100%|██████████| 81/81 [00:04<00:00, 18.21sample/s]


Total number of samples in the dataset is 29144.
Total number of classes in the dataset is 81.
Train dataset size is 22610 (images). Test dataset size is 6534 (images)


Converting to Yolo format:   0%|          | 0/81 [00:00<?, ?it/s]

Annotations for ee577821df67a483: [('Spider', 8.73472, 3.6795649999999998, 1019.5466240000001, 361.02515)]
Annotations for 8c922ad0ea78fecf: [('Spider', 76.703734, 36.61824, 532.717382, 1009.0270720000001)]
Annotations for 81067426068b965f: [('Spider', 237.22803199999998, 233.995892, 808.9753599999999, 408.2457600000001)]
Annotations for 6904fd6dbcad110c: [('Spider', 234.24409599999998, 0.343905, 949.015552, 680.545092)]
Annotations for 347221a32e893e8e: [('Spider', 140.133916, 54.025216, 526.0235759999999, 938.9199359999999)]
Annotations for d6dd405ff27a5dca: [('Spider', 179.59423999999999, 25.269634000000003, 881.7285119999999, 647.319397)]
Annotations for 9ef3ccac95c80fba: [('Spider', 179.06688, 69.91027199999999, 761.335808, 673.4845439999999)]
Annotations for 2389878e316820e7: [('Spider', 186.13145600000001, 94.319616, 818.615296, 705.41184)]
Annotations for 65704399bc54a790: [('Spider', 0.282624, 178.831872, 1023.527936, 712.708608)]
Annotations for 87e3c106fdb434b4: [('Spider', 

Converting to Yolo format:   1%|          | 1/81 [00:04<05:44,  4.30s/it]

...

Annotations for 31ccc29fc66e8415: [('Deer', 263.04, 35.239385, 1023.36, 679.1553929999999)]
Annotations for 99606eaef7120ad8: [('Deer', 291.84, 73.561212, 956.8, 686.360403)]
Annotations for b3cc6f8071ddc0f6: [('Deer', 188.8, 190.09991, 728.32, 614.46385)]
Annotations for bd8def869c3e3d88: [('Deer', 122.88, 172.160256, 675.84, 764.7997439999999)]
Annotations for fdcb5cb00b0e4c52: [('Deer', 0.0, 0.0, 1023.108096, 680.108571)]
Annotations for 6bbe5a6fa4cd238a: [('Deer', 298.000384, 165.99983999999998, 743.999488, 715.0002870000001)]
Annotations for 0c8bdd392e50daf8: [('Deer', 295.68, 62.090847000000004, 716.8, 682.3600289999999)]
Annotations for 748840ac74952f77: [('Deer', 28.16, 64.626084, 537.6, 522.125901)]
Annotations for ab4243d1ca3ee659: [('Turkey', 0.0, 30.079872, 938.88, 575.360064)]
Annotations for 184c043f7a844d2e: [('Turkey', 113.92, 0.0, 1023.36, 682.3593460000001)]
Annotations for 1ba8879712cd7ba2: [('Turkey', 122.24, 0.0, 981.12, 767.360256)]
Annotations for bd6c91acf57d167

Converting to Yolo format:  99%|█████████▉| 80/81 [08:09<00:04,  4.87s/it]

Annotations for bd2b17ef8d411bb7: [('Sea lion', 10.999808, 81.000086, 940.9996800000001, 705.000304)]
Annotations for 8e660cc701e0f415: [('Sea lion', 69.12, 0.0, 874.24, 584.754302)]
Annotations for 1342e735b93cc28c: [('Sea lion', 10.88, 76.110212, 1023.36, 516.783538)]
Annotations for 205c880717907a08: [('Sea lion', 256.64, 83.172986, 934.4, 610.3602830000001)]
Annotations for ea87e22457b6bc9f: [('Sea lion', 0.0, 161.459834, 509.44, 682.3593460000001), ('Sea lion', 588.16, 202.465105, 1023.36, 597.785505)]
Annotations for debf8c2711424b6f: [('Sea lion', 386.048, 210.77507999999997, 1022.976, 616.976592)]
Annotations for 89d5f49ef0a9d606: [('Sea lion', 265.6, 93.544363, 1022.08, 682.3593460000001)]
Annotations for 0b9132ad6f49f8ec: [('Sea lion', 234.24, 0.0, 1023.36, 682.3593460000001)]
Annotations for 22e06ab3e0d1fcde: [('Sea lion', 210.56, 3.19872, 946.56, 644.17692)]
Annotations for 8991d8549add6f62: [('Sea lion', 108.16, 0.0, 1023.36, 624.861072)]
Annotations for 60a3a338c2f9c835: 

Converting to Yolo format: 100%|██████████| 81/81 [08:13<00:00,  6.09s/it]

Annotations for 1bd4988852601bd9: [('Sea lion', 99.84, 135.190288, 975.36, 674.030161)]
{'path': '/kaggle/working/', 'train': 'train/images', 'val': 'test/images', 'nc': 81, 'names': ['Spider', 'Parrot', 'Scorpion', 'Sea turtle', 'Cattle', 'Fox', 'Hedgehog', 'Turtle', 'Cheetah', 'Snake', 'Shark', 'Horse', 'Magpie', 'Hamster', 'Woodpecker', 'Eagle', 'Penguin', 'Butterfly', 'Lion', 'Otter', 'Poacher', 'Raccoon', 'Hippopotamus', 'Bear', 'Chicken', 'Pig', 'Owl', 'Caterpillar', 'Koala', 'Polar bear', 'Squid', 'Whale', 'Harbor seal', 'Raven', 'Mouse', 'Tiger', 'Lizard', 'Ladybug', 'Red panda', 'Kangaroo', 'Starfish', 'Worm', 'Tortoise', 'Ostrich', 'Goldfish', 'Frog', 'Swan', 'Elephant', 'Sheep', 'Snail', 'Zebra', 'Moths and butterflies', 'Shrimp', 'Fish', 'Panda', 'Lynx', 'Duck', 'Jaguar', 'Goose', 'Goat', 'Rabbit', 'Giraffe', 'Crab', 'Tick', 'Monkey', 'Bull', 'Seahorse', 'Centipede', 'Mule', 'Rhinoceros', 'Canary', 'Camel', 'Brown bear', 'Sparrow', 'Squirrel', 'Leopard', 'Jellyfish', 'Croco

In [5]:
# Load a pretrained YOLO model (recommended for training)
model = YOLO('yolov8n.pt')

# Train the model using the processed dataset
results = model.train(
    data='config.yaml', 
    epochs=100 if not DEBUG else 1,
    optimizer='Adam',
    val=True,
    batch=64,
    imgsz=640,
    device=[0] if not CPU else 'cpu',
    lr0=0.001,
    lrf=0.0005
)

# Evaluate the model's performance on the validation set
results = model.val()

print(results)

100%|██████████| 6.23M/6.23M [00:00<00:00, 110MB/s]


New https://pypi.org/project/ultralytics/8.2.26 available 😃 Update with 'pip install -U ultralytics'
Ultralytics YOLOv8.1.15 🚀 Python-3.10.13 torch-2.1.2 CUDA:0 (Tesla P100-PCIE-16GB, 16276MiB)
engine/trainer: task=detect, mode=train, model=yolov8n.pt, data=config.yaml, epochs=100, time=None, patience=50, batch=64, imgsz=640, save=True, save_period=-1, cache=False, device=[0], workers=8, project=None, name=train, exist_ok=False, pretrained=True, optimizer=Adam, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False

100%|██████████| 755k/755k [00:00<00:00, 27.2MB/s]
2024-05-31 14:47:44,438	INFO util.py:124 -- Outdated packages:
  ipywidgets==7.7.1 found, needs ipywidgets>=8
Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.
2024-05-31 14:47:45,194	INFO util.py:124 -- Outdated packages:
  ipywidgets==7.7.1 found, needs ipywidgets>=8
Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.
2024-05-31 14:47:47.313874: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-05-31 14:47:47.313978: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-05-31 14:47:47.428709: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS facto

Overriding model.yaml nc=80 with nc=81

                   from  n    params  module                                       arguments                     
  0                  -1  1       464  ultralytics.nn.modules.conv.Conv             [3, 16, 3, 2]                 
  1                  -1  1      4672  ultralytics.nn.modules.conv.Conv             [16, 32, 3, 2]                
  2                  -1  1      7360  ultralytics.nn.modules.block.C2f             [32, 32, 1, True]             
  3                  -1  1     18560  ultralytics.nn.modules.conv.Conv             [32, 64, 3, 2]                
  4                  -1  2     49664  ultralytics.nn.modules.block.C2f             [64, 64, 2, True]             
  5                  -1  1     73984  ultralytics.nn.modules.conv.Conv             [64, 128, 3, 2]               
  6                  -1  2    197632  ultralytics.nn.modules.block.C2f             [128, 128, 2, True]           
  7                  -1  1    295424  ultralytic

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

...

                   all       6032       7618      0.697      0.625      0.689      0.584
                Spider       6032        213      0.921      0.948      0.969      0.833
                Parrot       6032        203      0.925      0.798      0.904       0.79
              Scorpion       6032         44       0.89      0.773      0.902       0.78
            Sea turtle       6032        104      0.783      0.243      0.625      0.534
                Cattle       6032        209      0.904      0.269      0.636      0.556
                   Fox       6032         72      0.768      0.833      0.879      0.786
              Hedgehog       6032         52      0.933       0.75      0.897      0.768
                Turtle       6032          7          0          0     0.0219     0.0112
               Cheetah       6032         39       0.85      0.513      0.729      0.639
                 Snake       6032        219      0.846      0.904      0.922      0.733
                 Shar

lr/pg0,█▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
lr/pg1,▃███▇▇▇▇▇▇▆▆▆▆▆▅▅▅▅▅▅▄▄▄▄▄▃▃▃▃▃▃▂▂▂▂▂▁▁▁
lr/pg2,▃███▇▇▇▇▇▇▆▆▆▆▆▅▅▅▅▅▅▄▄▄▄▄▃▃▃▃▃▃▂▂▂▂▂▁▁▁
metrics/mAP50(B),▁▂▄▅▅▆▆▇▇▇▇▇████████████████████████████
metrics/mAP50-95(B),▁▂▃▄▅▅▆▆▆▇▇▇▇▇▇█████████████████████████
metrics/precision(B),▅▁▁▃▄▄▅▅▆▆▇▇▇▇▇▇▇▇▇▇▇▇▇▇█▇▇▇▇▇▇▇▇█▇██▇▇█
metrics/recall(B),▁▃▅▅▅▆▆▆▇▇▇▇▇███████████████████████████
model/GFLOPs,▁
model/parameters,▁
model/speed_PyTorch(ms),▁
train/box_loss,▇█▇▆▆▆▆▆▅▅▅▅▅▅▅▅▅▅▅▄▄▄▄▄▄▄▄▄▄▄▄▃▃▃▃▃▂▁▁▁


Ultralytics YOLOv8.1.15 🚀 Python-3.10.13 torch-2.1.2 CUDA:0 (Tesla P100-PCIE-16GB, 16276MiB)
Model summary (fused): 168 layers, 3160775 parameters, 0 gradients, 8.8 GFLOPs


val: Scanning /kaggle/working/test/labels.cache... 6032 images, 0 backgrounds, 0 corrupt: 100%|██████████| 6032/6032 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 95/95 [00:56<00:00,  1.67it/s]


                   all       6032       7618      0.703      0.624      0.689      0.584
                Spider       6032        213      0.922      0.948      0.969      0.838
                Parrot       6032        203       0.92      0.791      0.903      0.789
              Scorpion       6032         44      0.919      0.775      0.907      0.786
            Sea turtle       6032        104      0.778      0.236      0.627      0.534
                Cattle       6032        209      0.918      0.269      0.637      0.556
                   Fox       6032         72      0.786      0.833      0.876      0.781
              Hedgehog       6032         52      0.957       0.75      0.899      0.768
                Turtle       6032          7          0          0     0.0218     0.0112
               Cheetah       6032         39       0.86      0.513      0.728      0.639
                 Snake       6032        219      0.849      0.901      0.924      0.735
                 Shar

In [6]:
!zip -r results.zip /kaggle/working/runs

  adding: kaggle/working/runs/ (stored 0%)
  adding: kaggle/working/runs/detect/ (stored 0%)
  adding: kaggle/working/runs/detect/train2/ (stored 0%)
  adding: kaggle/working/runs/detect/train2/PR_curve.png (deflated 7%)
  adding: kaggle/working/runs/detect/train2/val_batch0_pred.jpg (deflated 12%)
  adding: kaggle/working/runs/detect/train2/val_batch2_labels.jpg (deflated 7%)
  adding: kaggle/working/runs/detect/train2/confusion_matrix.png (deflated 16%)
  adding: kaggle/working/runs/detect/train2/val_batch1_labels.jpg (deflated 6%)
  adding: kaggle/working/runs/detect/train2/val_batch1_pred.jpg (deflated 6%)
  adding: kaggle/working/runs/detect/train2/R_curve.png (deflated 9%)
  adding: kaggle/working/runs/detect/train2/P_curve.png (deflated 8%)
  adding: kaggle/working/runs/detect/train2/confusion_matrix_normalized.png (deflated 16%)
  adding: kaggle/working/runs/detect/train2/F1_curve.png (deflated 9%)
  adding: kaggle/working/runs/detect/train2/val_batch2_pred.jpg (deflated 6%)
  

In [7]:
!ls

config.yaml  results.zip  runs	test  train  wandb  yolov8n.pt


In [8]:
from IPython.display import FileLink
FileLink(r'results.zip')

/kaggle/working/results.zip